In [ ]:
import dataclasses
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nl
import scipy.signal as signal



# Example: Vehicle Dynamics in Classes



## Benefit of using python in simulating the vehicle dynamics



* May save cost before establishing a more solid research basis capable of accomodating propriotary simulation software
* Possibly smoother integration with existing and future libraries or modules of python extending functionalities



## Formulation



Frequently Vehicle dynamics is described as follows.



$$
\frac{d}{dt} 
    \begin{bmatrix} 
        v \\ 
        r 
    \end{bmatrix} =  \begin{bmatrix} 
         \frac{-\left(C_{af} + C_{ar}\right)}{mu} & \frac{bC_{ar} - a C_{af}}{mu} -u \\
         \frac{bC_{ar} - a C_{af}}{I_z u}  &  \frac{-\left(a^2 C_{af} + b^2 C_{ar}\right)}{I_z u} \\
     \end{bmatrix}     \begin{bmatrix} v \\ r \end{bmatrix} + \begin{bmatrix} 
           \frac{C_{af}}{m} \\ 
           \frac{a C_{af}}{I_{z}} \\ 
     \end{bmatrix} \delta_f
 $$



Following class would prepare the matrices of state matrix.



In [ ]:
@dataclasses.dataclass
class BicycleModel:
    m_kg:float = 1500
    Iz_kg_m2:float = 2420
    u_mps:float = 20
    a_m:float = 1.4
    b_m:float = 2.54 - 1.4
    Caf:float = 44000 * 2
    Car:float = 47000 * 2

    def get_a00(self):
        return -(self.Caf + self.Car) / (self.m_kg * self.u_mps)

    def get_a01(self):
        return (self.b_m * self.Car - self.a_m * self.Caf) / (self.m_kg * self.u_mps) - self.u_mps

    def get_a10(self):
        return (self.b_m * self.Car - self.a_m * self.Caf) / (self.Iz_kg_m2 * self.u_mps)

    def get_a11(self):
        return -((self.a_m ** 2) * self.Caf + (self.b_m**2) * self.Car) / (self.Iz_kg_m2 * self.u_mps)

    def get_mat_a(self):
        """
        Calculate state matrix of linear model
        """
        return np.array([
            [self.get_a00(), self.get_a01()],
            [self.get_a10(), self.get_a11()],
        ])

    def get_b00(self):
        return self.Caf / self.m_kg

    def get_b10(self):
        return self.a_m * self.Car / self.Iz_kg_m2

    def get_mat_b(self):
        """
        Calculate input matrix of linear model
        """
        return np.array([
            [self.get_b00()],
            [self.get_b10()],
        ])



Following cell creates an object of the `bicycle_model` class and checks if it can create the A & B matrices.



In [ ]:
vehicle = BicycleModel()
assert isinstance(vehicle.get_mat_a(), np.ndarray)
assert isinstance(vehicle.get_mat_b(), np.ndarray)



## Available numerical solvers for ordinary differential equations



Two of the possible options to numerically solve an ordianry differential equations are as follows.



* manual implementation
* `scipy` module



### Scipy.integrate

Roughly, there are two interfaces to numerical solvers for ordinary differential equations.



* `scipy.integrate.solve_ivp`
* `scipy.integrate.OdeSolver`



Following classes are to assist the numerical solvers.



* `scipy.integrate.DenseOutput`
* `scipy.integrate.OdeSolution`



### Scipy.signal



Scipy also has a subpackage `signal` with following functions, comparable to those of the  Matlab$^{TM}$.



* `scipy.signal.lsim`
* `scipy.signal.impulse`
* `scipy.signal.step`
* `scipy.signal.dlsim`
* `scipy.signal.dimpulse`
* `scipy.signal.dstep`



Functions with names starting with `d`are for discrete time.



Following classes are available in the `scipy.signal`.



* `scipy.signal.lti`
* `scipy.signal.dlti`
* `scipy.signal.StateSpace`
* `scipy.signal.TransferFunction`
* `scipy.signal.ZerosPolesGain`



Following is one possible example.



In [ ]:
class BicycleModelSimulator(signal.StateSpace):
    def __init__(self, model:BicycleModel):
        self.model = model
        self.mat_a = self.model.get_mat_a()
        self.mat_b = self.model.get_mat_b()
        self.mat_c = np.identity(self.mat_a.shape[0])
        self.mat_d = np.zeros((self.mat_a.shape[0], self.mat_b.shape[1]))
        
        super(BicycleModelSimulator, self).__init__(self.mat_a, self.mat_b, self.mat_c, self.mat_d,)



In [ ]:
simulator = BicycleModelSimulator(BicycleModel())



We can prepare the input signal as follows.



In [ ]:
t_sec_array = np.arange(0, 6, 0.01)
steering_deg_array = 0.5 * np.sin((0.4 * 2 * np.pi) * t_sec_array)
steering_rad_array = np.deg2rad(steering_deg_array)



### python-control

Another possible option is `python-control`.  This package is also actively developed by enthusiastic open source community.



## Visualization



Frequently `matplotlib` is used to plot data in two dimensional spaces.



## Slope field of a two dimensional bicycle model



With the state matrix $A$, we can plot the slope vector $\frac{d}{dt}\begin{bmatrix}v & r \end{bmatrix} ^T$ at all values of  $\begin{bmatrix}v & r \end{bmatrix}^T$.



$$
\dot x_{2 \times 1} = A_{2 \times 2} x_{2 \times 1}
$$



$$
\dot x_{2 \times m} = A_{2 \times 2} x_{2 \times m}
$$



In [ ]:
v_mps = np.linspace(-1, 1)
r_dps = np.linspace(-5, 5) # degree per second
r_rps = np.deg2rad(r_dps)

v_grid_mps, r_grid_rps = np.meshgrid(v_mps, r_dps)
v_grid_mps.shape, r_grid_rps.shape



In [ ]:
v_row = v_grid_mps.flatten()
r_row = r_grid_rps.flatten()



In [ ]:
v_r_grid = np.array([v_row, r_row])
v_r_grid.shape



In [ ]:
vehicle = BicycleModel()
mat_A = vehicle.get_mat_a()
mat_A.shape



In [ ]:
x_dot_grid = mat_A @ v_r_grid
x_dot_grid.shape



In [ ]:
v_dot_grid_m = x_dot_grid[0, :].reshape(v_grid_mps.shape)
r_dot_grid_rad = x_dot_grid[1, :].reshape(r_grid_rps.shape)
v_dot_grid_m.shape, r_dot_grid_rad.shape



In [ ]:
plt.figure(figsize=(10, 10))
plt.quiver(v_mps, r_dps, v_dot_grid_m, np.rad2deg(r_dot_grid_rad))
plt.grid(True)
plt.xlabel('v (m/s)')
plt.ylabel('r (deg/s)')
plt.savefig('bicycle_model_slope_fields.svg')



## Utilizing `pandas` to handle input and output arrays



To handle time series data, a researcher may consider utilizing `pandas`.



## References



* Johansson, R., "Ordinary Differential Equations" in Numerical Python: Scientific Computing and Data Science Applications with Numpy, Scipy, and Matplotlib, APress, 2019, pp. 347-388.
* The SciPy community, "Integration and ODEs (scipy.integrate)" in API Reference, SciPy v1.4.1 Reference Guide, 2019. [Online] Available: https://docs.scipy.org/doc/scipy/reference/integrate.html.
* The SciPy community, "Signal processing (scipy.signal)" in API Reference, SciPy v1.4.1 Reference Guide, 2019. [Online] Available: https://docs.scipy.org/doc/scipy/reference/signal.html.
* Python Control Systems Library community, "Python Control Systems Library", 2019. [Online] Available: https://python-control.readthedocs.io/en/0.8.3/.



## Final Bell



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

